Import packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

## The Backward Euler method

We analyze the semi-batch reactor. The reactor model contains a non-autonomous term (right hand-side contains the independent variable $t$ explicitly).

$\frac{dC_A}{dt}=\frac{\dot{V}}{V_0+\dot{V}t}\left(C_A^{in}-C_A\right)-kC_A$

$\frac{dC_B}{dt}=kC_A$

Initial contitions:

$C_A(t=0\,s)=1\, mol/L,\; C_B(0)=0\, mol/L$


Define parameters.

In [ ]:
# time domain
t_start = 0.0
t_end = 30.0
n = 500
t = np.linspace(t_start, t_end, n)

# initial conditions
c0 = np.array([1,0])

Define chemical reaction system.

In [ ]:
def dC(t: np.ndarray, c: np.ndarray) -> np.ndarray:
    """ODE system for semi-batch reactor.

    Parameters
    ----------
    t : np.ndarray
        Time variable
    c : np.ndarray
        Concentration of individual components

    Returns
    -------
    np.ndarray
        Concentration gradient in reactor
    """
    cA, _ = c
    k = 0.2
    Vdot = 0.1
    V_0 = 10
    cA_in = 0.5
    dcA = Vdot/(V_0+Vdot*t)*(cA_in-cA)-k*cA
    dcB = k*cA
    return np.array([dcA, dcB])

Define backward euler method.

In [ ]:
def backward_euler_fixed_point(func: Callable, y0: np.ndarray, t: np.ndarray,
                               tol:float=1e-6, max_iter:int=100) -> np.ndarray:
    """Solves the ODE y' = f(t, y) using the Backward (Implicit) Euler method
    with fixed-point iteration.

    Parameters
    ----------
    func : Callable
        Function that defines the ODE (y' = func(t, y)).
    y0 : np.ndarray
        Initial condition.
    t : np.ndarray
        Time domain.
    tol : float, optional
        Tolerance for the fixed-point iteration (default is 1e-6).
    max_iter : int, optional
        Maximum number of iterations for the fixed-point iteration (default is 100).

    Returns
    -------
    np.ndarray
        Array of solution values at the time points.
    """
    # Initialize arrays for time and solution values
    y = np.zeros([len(y0),len(t)])
    y[:,0] = y0
    dt = t[1]-t[0]

    # Iterate over each time step
    for n in range(len(t)-1):
        # Initial guess for y_{i+1}
        y_next = y[:,n]
        
        # Fixed-point iteration to solve for y_{i+1}
        for i in range(max_iter):
            y_new = y[:,n] + dt * func(t[n+1], y_next)
            if max(np.abs(y_new - y_next)) < tol:
                break
            y_next = y_new
        
        # Update solution
        y[:,n+1] = y_next
    
    return y

Execute backward euler method.

In [ ]:
c = backward_euler_fixed_point(dC, c0, t)

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, c[0,:], label = "cA")
ax.plot(t, c[1,:], label = "cB")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("The Backward Euler method")
ax.legend()
ax.grid()
fig.show()